### Preliminary setup

In [82]:
import pandas as pd             # For data transformation
import pyasn1
asndb = pyasn.pyasn('ipasn_20140513.dat')

In [111]:
# input csv
input_file = "./benign_flow/test8.csv"
df = pd.read_csv(input_file)

column_names = ["DURATION", "SRC_IP", "DST_IP", "SRC_PORT", "DST_PORT", "QUIC_VERSION", "BYTES", "PROTOCOL"]
df.columns = column_names

# Specify the column name and the string to filter out
column_name = "PROTOCOL"  # Replace with the actual column name
protocol_filter = "QUIC"  # Replace with the string to remove

# Filter out rows containing the specific string in the specified column
df = df[df[column_name].str.contains(protocol_filter, na=False)]

df["BYTES"] = pd.to_numeric(df["BYTES"], errors='coerce').fillna(0)
df["DURATION"] = df["DURATION"] - df["DURATION"].iloc[0]

In [112]:
# source IP
ipsrc = df["SRC_IP"].iloc[0]
portsrc = df["SRC_PORT"].iloc[0]
sni = "youtube.com"
cat = "Streaming"

flow = []

df["true_dest"] = df.apply(lambda row: row["DST_IP"] if row["SRC_IP"] == ipsrc else row["SRC_IP"], axis=1)
df["group"] = (df["true_dest"] != df["true_dest"].shift()).cumsum()

for group, group_df in df.groupby("group"):
    num_subgroups = (len(group_df) + 29) // 30
    subgroups = [group_df.iloc[i * 30:(i+1) * 30] for i in range(num_subgroups)]
    
    # print(f"Group {group}:")
    # print(group_df)

    for  subgroup in subgroups:
        ppi_dir = []
        ipdst = subgroup["true_dest"].iloc[0]  # The unique normalized destination for this subgroup
        portdst = subgroup["DST_PORT"].iloc[0] 

        ppi_time = [0]
        ppi_size = [int(subgroup["BYTES"].iloc[0])]
        for i in range(1, len(subgroup)):
            # Calculate the time difference between consecutive packets
            duration = int((subgroup["DURATION"].iloc[i] - subgroup["DURATION"].iloc[i - 1]) * 1000)
            ppi_time.append(duration)
            ppi_size.append(int(subgroup["BYTES"].iloc[i]))



        for _, row in subgroup.iterrows():
            if row["SRC_IP"] == ipsrc and row["DST_IP"] == ipdst:
                ppi_dir.append(1)
            elif row["SRC_IP"] == ipdst  and row["DST_IP"] == ipsrc:
                ppi_dir.append(-1)
            else:
                ppi_dir.append(0)

            bytes_fromsrc = int(subgroup.loc[(subgroup["SRC_IP"] == ipsrc) & (subgroup["DST_IP"] == ipdst),"BYTES"].sum())
            bytes_rev = int(subgroup.loc[(subgroup["SRC_IP"] == ipdst) & (subgroup["DST_IP"] == ipsrc),"BYTES"].sum())
            packets = int(subgroup.loc[(subgroup["SRC_IP"] == ipsrc) & (subgroup["DST_IP"] == ipdst),"BYTES"].count())
            packets_rev = int(subgroup.loc[(subgroup["SRC_IP"] == ipdst) & (subgroup["DST_IP"] == ipsrc),"BYTES"].count())
            ppi_len = len(ppi_dir)

            ppi_rtt = 0
            in_group = False

            # Iterate through the list
            for value in ppi_dir:
                if value == -1:  # Start or continue a group of -1's
                    if not in_group:
                        in_group = True  # Beginning of a group
                elif value == 1:  # Start or continue a group of 1's
                    if not in_group:
                        in_group = True  # Beginning of a group
                    elif in_group:
                        ppi_rtt += 1
                        in_group = False  # Reset for the next group

            # Handle the case where the list ends with a valid pair
            if in_group:
                ppi_rtt += 1

        # print("Subgroup:")
        # print(subgroup)

        dur = round(subgroup["DURATION"].max() - subgroup["DURATION"].min(), ndigits=6)

        flow.append([ipsrc, ipdst, asndb.lookup(ipdst)[0], portsrc, portdst, 1, sni, dur, bytes_fromsrc, bytes_rev, packets, packets_rev, ppi_len, ppi_rtt, cat, [ppi_time, ppi_dir, ppi_size]])

In [113]:
# Read the existing CSV file
file_path = "./benign_flow/benign.csv"
df = pd.read_csv(file_path)

# Append each item in the flow list as a new row in the DataFrame
for item in flow:
    df.loc[len(df)] = item

# Write the updated DataFrame back to the CSV file
df.to_csv(file_path, index=False)

C:\Users\John Raphael Mundo\AppData\Local\Temp\ipykernel_27116\1256721800.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)] = item
C:\Users\John Raphael Mundo\AppData\Local\Temp\ipykernel_27116\1256721800.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)] = item
C:\Users\John Raphael Mundo\AppData\Local\Temp\ipykernel_27116\1256721800.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, th

In [109]:
for item in flow:
    print(item)

['142.250.197.74', '10.10.3.10', None, 443, 40877, 1, 'youtube.com', 2.199841, 9522, 3606, 17, 11, 30, 11, 'Streaming', [[0, 0, 0, 0, 0, 11, 0, 0, 0, 1, 150, 1, 164, 0, 0, 23, 6, 180, 0, 0, 2, 0, 164, 18, 0, 2, 26, 163, 1277, 0], [1, 1, 1, 1, -1, 1, 1, 1, -1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, 1, -1, -1, 1, 0, 0], [40, 1250, 1250, 1250, 1250, 1250, 1250, 1250, 39, 40, 0, 0, 956, 122, 31, 24, 381, 282, 21, 35, 1246, 517, 27, 278, 248, 35, 32, 24, 1250, 1250]]]
['142.250.197.74', '142.250.199.246', 15169, 443, 38404, 1, 'youtube.com', 0.000456, 0, 0, 0, 0, 4, 0, 'Streaming', [[0, 0, 0, 0], [0, 0, 0, 0], [40, 1250, 1250, 1250]]]
['142.250.197.74', '10.10.3.10', None, 443, 443, 1, 'youtube.com', 0.0, 0, 0, 0, 0, 1, 0, 'Streaming', [[0], [0], [1250]]]
['142.250.197.74', '142.250.199.246', 15169, 443, 38404, 1, 'youtube.com', 5.5e-05, 0, 0, 0, 0, 3, 0, 'Streaming', [[0, 0, 0], [0, 0, 0], [1250, 1250, 1250]]]
['142.250.197.74', '10.10.3.10', None, 443, 443, 1, 'youtube.com', 

In [61]:
# Save the filtered DataFrame to a new CSV file
output_file = "./benign_flow/filtered_output.csv"
df.to_csv(output_file, index=False)